## Login to Hugging Face

In [1]:
from dotenv import load_dotenv
import os
from huggingface_hub import login

load_dotenv()
token = os.getenv("HUGGINGFACE_TOKEN")
login(
    token=token, # ADD YOUR TOKEN HERE
    add_to_git_credential=True
)

Token is valid (permission: write).
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /home/pathfinder/.cache/huggingface/token
Login successful


## Imports

In [2]:
from IPython.display import display, Markdown

# pytorch
import torch

# huggingface
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig
)

## Device

In [3]:
# Device setup
device = (
    "cuda:0" if torch.cuda.is_available() else # Nvidia GPU
    "mps" if torch.backends.mps.is_available() else # Apple Silicon GPU
    "cpu"
)
print(f"Device = {device}")

Device = cuda:0


In [4]:
# Flash Attention Implementation
if device == "cuda:0":
    if torch.cuda.get_device_capability()[0] >= 8: # Ampere, Ada, or Hopper GPUs
        attn_implementation = "flash_attention_2"
        torch_dtype = torch.bfloat16
    else:
        attn_implementation = "eager"
        torch_dtype = torch.float16
else:
    attn_implementation = "eager"
    torch_dtype = torch.float32
print(f"Attention Implementation = {attn_implementation}")

Attention Implementation = flash_attention_2


## Hyperparameters

In [5]:
################################################################################
# Tokenizer parameters
################################################################################
max_length=8192
padding="do_not_pad" # "max_length", "longest", "do_not_pad"
truncation=True

################################################################################
# Generation parameters
################################################################################
num_return_sequences=1
max_new_tokens=1024
do_sample=True # True for sampling, False for greedy decoding
temperature=0.6
top_k=0 # not recommended
top_p=0.9
repetition_penalty=1.1

################################################################################
# bitsandbytes parameters
################################################################################
load_in_4bit=True
bnb_4bit_compute_dtype=torch_dtype
bnb_4bit_quant_type="nf4" # "nf4", #fp4"
bnb_4bit_use_double_quant=True

## Model

In [6]:
# Model List

# gemma variants
# "google/gemma-1.1-7b-it"
# "google/codegemma-7b-it"

# llama variants
# "meta-llama/Meta-Llama-3-8B" // downloaded
# "meta-llama/Meta-Llama-3-8B-Instruct" // downloaded
# "codellama/CodeLlama-7b-Instruct-hf"
# "PathFinderKR/Waktaverse-Llama-3-KO-8B-Instruct"

# mistral variants
# "mistralai/Mistral-7B-Instruct-v0.2"

# openELM variants
# "apple/OpenELM-3B-Instruct" // downloaded

# solar variants
# "upstage/SOLAR-10.7B-Instruct-v1.0" // downloaded
# "PathFinderKR/Waktaverse-SOLAR-KO-10.7B-Instruct"

In [7]:
model_id = "PathFinderKR/Waktaverse-Llama-3-KO-8B-Instruct"

In [8]:
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.padding_side = "right"

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [9]:
quantization_config = BitsAndBytesConfig(
    load_in_4bit=load_in_4bit,
    bnb_4bit_compute_dtype=bnb_4bit_compute_dtype,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_use_double_quant=bnb_4bit_use_double_quant
)

In [10]:
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map=device,
    attn_implementation=attn_implementation,
    torch_dtype=torch_dtype,
    quantization_config=quantization_config
)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [11]:
# Display the model architecture
display(Markdown(f'```{model}```'))

```LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaFlashAttention2(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm()
  )
  (lm_head): Linear(in_features=4096, out_features=128256, bias=False)
)```

## Inference

In [12]:
def generate_response(system ,user):
    messages = [
        {"role": "system", "content": system},
        {"role": "user", "content": user}
    ]
    prompt = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=False
    )
    
    input_ids = tokenizer.encode(
        prompt,
        max_length=max_length,
        padding=padding,
        truncation=truncation,
        add_special_tokens=False,
        return_tensors="pt"
    ).to(device)
    
    outputs = model.generate(
        input_ids=input_ids,
        pad_token_id=tokenizer.eos_token_id,
        num_return_sequences=num_return_sequences,
        max_new_tokens=max_new_tokens,
        do_sample=do_sample,
        temperature=temperature,
        top_k=top_k,
        top_p=top_p,
        repetition_penalty=repetition_penalty
    )
    
    return tokenizer.decode(outputs[0], skip_special_tokens=False)

In [13]:
system_prompt = "다음 지시사항에 대한 응답을 작성해주세요."

In [14]:
user_prompt = "피보나치 수열에 대해 설명해주세요."

In [15]:
response = generate_response(system_prompt, user_prompt)
print(response)

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

다음 지시사항에 대한 응답을 작성해주세요.<|eot_id|><|start_header_id|>user<|end_header_id|>

피보나치 수열에 대해 설명해주세요.<|eot_id|><|start_header_id|>assistant<|end_header_id|>"

피보나치 수열은 1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144와 같은 순서로 숫자가 이어지는 수열입니다. 이 수열의 이름은 이탈리아의 수학자 피보나치에서 따온 것입니다. 피보나치 수열은 각 항이 이전 두 항의 합인 특징이 있습니다. 예를 들어, 첫 번째 항과 두 번째 항은 모두 1이며, 세 번째 항부터는 이전 두 항의 합이 됩니다.

피보나치 수열에는 몇 가지 속성이 있습니다:

1. 각 항은 이전 두 항의 합이라는 점.
2. 피보나치 수열은 무한히 계속됩니다.
3. 피보나치 수열은 기하급수라는 특성을 가지고 있습니다. 즉, 각 항은 이전 항의 배수보다 더 커지며, 이는 피보나치 수열의 성장속도가 급격하게 증가합니다.
4. 피보나치 수열은 각 항이 이전 두 항의 합이라는 점에서 유도된 등차수열입니다.

피보나치 수열은 수학적 개념뿐만 아니라 실생활에서도 자주 사용됩니다. 예를 들어, 금융 시장에서는 피보나치 수열을 기반으로 한 주식 시세예측 모델이 사용되기도 합니다. 또한, 컴퓨터 과학 분야에서도 피보나치 수열을 이용하여 알고리즘을 설계하기도 합니다.

피보나치 수열은 다음과 같이 계산할 수 있습니다:

1. 첫 번째 항: 0 또는 1
2. 두 번째 항: 1
3. n번째 항: (n-1)번째 항 + (n-2)번째 항

피보나치 수열은 유명한 수학적 개념 중 하나이며, 수학적 이론 뿐만 아니라 실무에서도 널리 활용되고 있습니다. 피보나치 수열을 통해 수학적 개념을 이해하고, 실용적인 문제를 해결하는 데 도움이 될 수 있습니다.<|eot_id|>


## Multi-turn Conversation

In [16]:
system_prompt = "다음 지시사항에 대한 응답을 작성해주세요."
conversation_history = [{"role": "system", "content": system_prompt}]

In [17]:
def generate_conversation(user):
    global conversation_history
    conversation_history.append({"role": "user", "content": user})
    prompt = tokenizer.apply_chat_template(
        conversation_history,
        tokenize=False,
        add_generation_prompt=False
    )
    
    input_ids = tokenizer.encode(
        prompt,
        max_length=max_length,
        padding=padding,
        truncation=truncation,
        add_special_tokens=False,
        return_tensors="pt"
    ).to(device)
    
    outputs = model.generate(
        input_ids=input_ids,
        pad_token_id=tokenizer.eos_token_id,
        num_return_sequences=num_return_sequences,
        max_new_tokens=max_new_tokens,
        do_sample=do_sample,
        temperature=temperature,
        top_k=top_k,
        top_p=top_p,
        repetition_penalty=repetition_penalty
    )
    
    assistant_response = tokenizer.decode(outputs[0], skip_special_tokens=False).split("<|start_header_id|>assistant<|end_header_id|>")[-1]
    conversation_history.append({"role": "assistant", "content": assistant_response})
    return assistant_response

In [18]:
user_input = "이번 주말에 할 수 있는 취미 활동을 추천해주세요."
response = generate_conversation(user_input)
print(response)



여러가지 취미 활동 중에서 가장 적합한 것을 추천해드리겠습니다.

**1. 낚시**: 날씨가 좋아서 낚시를 즐길 수 있어요! 물고기를 잡으면 식사에도 활용할 수 있고, 자연과 함께하는 시간이 좋습니다.
**2. 캠핑**: 가족과 함께 캠핑을 떠날 수 있습니다. 야외에서 보이는 별들만큼 특별한 경험을 해보세요!
**3. 요리**: 재료를 준비하고 다양한 레시피를 시도해보고 맛있는 음식을 만들 수 있습니다. 
**4. 독서**: 책을 읽는 것은 정신적인 안정감을 가져다 줄 뿐 아니라 새로운 지식과 정보를 얻을 수도 있습니다.
**5. 사진-taking**: 카메라를 가지고 산책을 하거나 여행을 가서 좋은 순간을 찍어보세요. 
**6. 운동**: 건강을 챙기면서 스트레스를 풀 수 있는 방법입니다. 달리기, 자전거 타기, 수영 등 다양한 옵션이 있어요.
**7. 여행**: 가까운 곳으로 여행을 떠나 새롭고 아름다운 장소를 발견할 수 있습니다.

이러한 활동을 통해 당신의 취미를 찾아내거나 새로운 것을 시도해보세요! 

이 외에도 다양한 취미가 있으니, 무엇에 관심이 있다면 알려주시면 자세히 설명해드리겠습니다. 

즐거운 주말 보내세요!<|eot_id|>


In [19]:
user_input = "추천한 취미 활동 중에서 건강에 도움이 되는 것은 무엇인가요?"
response = generate_conversation(user_input)
print(response)



저는 **운동**을 추천했습니다. 운동은 몸과 마음을 건강하게 하는 데 큰 역할을 합니다. 단순히 신체를 움직이는 것뿐만 아니라, 스트레스를 줄이고 기분좋게 만들어주며, 인지능력 향상에도 도움을 줍니다. 또한, 운동을 통해 사회성을 높일 수도 있습니다!

또 다른 선택지는 **독서**입니다. 독서는 생각을 정리하고, 창의력을 키우며, 새로운 정보와 지식을 얻을 수 있습니다. 이러한 효과로 인해, 머리가 깔끔해지고 집중력이 증가합니다. 

물론, 이 외에도 다양한 건강에 도움이 되는 취미들이 있습니다. 어떤 것을 선호하실까요? 

(참고: 운동이나 독서 같은 취미를 선호한다면, 매일 30분 정도씩 시작하시면 좋습니다.)<|eot_id|>
